<a href="https://colab.research.google.com/github/SilahicAmil/NLP-NLTK/blob/main/Tweet_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweet Sentiment Analysis

In [2]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Tesnorflow
import tensorflow as tf



#Keras
import keras
from tensorflow.keras import layers
from keras.layers import TextVectorization
from tensorflow.keras import losses
from keras import callbacks
from keras.layers import Embedding

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Tweet_Sentiment/tweet-sentiment-extraction/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Tweet_Sentiment/tweet-sentiment-extraction/test.csv")

In [5]:
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


## Preprocessing

In [6]:
train_df = train_df.drop("selected_text", 1)

In [7]:
train_df.head()

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


In [8]:
# Dropping all neutral values
train_df = train_df[train_df.sentiment != "neutral"]
# Replacing pos and neg with numerical values
train_df_copy = train_df.copy()
mapping = {'positive': 1, 'negative': 0}

train_df_copy = train_df_copy.replace({'sentiment': mapping})

In [9]:
train_df_copy.head()

,textID,text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0
2,088c60f138,my boss is bullying me...,0
3,9642c003ef,what interview! leave me alone,0
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,1


Postivie =1

Negative = 0

## Shuffle Data

In [10]:
# Shuffle df
shuffled_train_df = train_df_copy.sample(frac=1, random_state=42)
shuffled_train_df.head()

,textID,text,sentiment
14813,508435b32d,thanks will try to behave,1
18134,de3833a81d,Today is lame because I am not in Orlando I a...,0
9074,85a65f0656,hahahaha! i laughed my **** off just now. thanks,1
23126,d70ed4038d,Gahh ! This weather sucksss !,0
16935,1b4d9119e9,Fallen in love with enter shikari again. Might...,1


In [11]:
# Test data
test_df.head()
test_df = test_df.drop("sentiment", 1)


In [12]:
test_df.head()

,textID,text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh
1,96d74cb729,Shanghai is also really exciting (precisely -...
2,eee518ae67,"Recession hit Veronique Branquinho, she has to..."
3,01082688c6,happy bday!
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!


In [13]:
train_df_copy.sentiment.value_counts()

1    8582
0    7781
Name: sentiment, dtype: int64

In [14]:
len(train_df_copy), len(test_df)

(16363, 3534)

## Validation Set Creation

In [15]:
# Creating val set

train_sentences, val_sentences, train_labels, val_labels = train_test_split(shuffled_train_df["text"].to_numpy(),
                                                                            shuffled_train_df["sentiment"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [16]:
len(train_sentences), len(train_labels)

(14726, 14726)

In [17]:
len(val_sentences), len(val_labels)

(1637, 1637)

In [18]:
train_sentences[:10], train_labels[:10]

(array(['Great find   http://amanita-design.net/samorost-1/',
        ' everyone loves u sarah not just the tweeters! the today show couldnt stop raving about your beautifulness...',
        'Hello dark hair! Well, my plans for today just got cancelled  make some better ones?!',
        'just watched devil wears prada, and i want to live in new york city more than ever. why must i be so poor',
        'Is feeling really bad about goofin` on  not knowing she really wasn`t at the meeting! Dang I`m sorry! Me & my big `ol mouth',
        'im blonding ... slowly but surely im blonding ... i wana scratch my hair but im not allowed',
        ' man i need 2 find a siitter Val still aint **** wit me  LOL',
        ' its my birthday too  but its the 10th of may right now for me.. and its probably the 9th for you.. happy birthday anyway',
        'Good morning all! Its a fabulous morning here!', '  nice one'],
       dtype=object), array([0, 1, 0, 0, 0, 0, 0, 1, 1, 1]))

## Converting Text to Numbers

Standarize each sample (lowercase + punctucation stripping)

Split each sample into substrings (usually words)

Recombine substrings to tokens (usually ngrams)

Index tokens (unique int value > Token)

Transform each sample with index into vector or dense float vector



In [19]:
# Find average number of tokens 
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

13.318823848974603

In [20]:
# Setting text vectorization variable
MAX_VOCAB_LEN = 10000 # Max words in vocab
MAX_LEN = 13 # MAX LEN OF SEQUENCE

txt_vect = TextVectorization(max_tokens=MAX_VOCAB_LEN,
                              output_mode="int",
                              output_sequence_length=MAX_LEN)

In [21]:
# Fitting txt_vect to train sentences (only do this on train)

txt_vect.adapt(train_sentences)

In [22]:
# Sample sentences and tokenize it

sample_sent = "you suck balls lol"
txt_vect([sample_sent])

<tf.Tensor: shape=(1, 13), dtype=int64, numpy=
array([[   8,  478, 1609,   64,    0,    0,    0,    0,    0,    0,    0,
           0,    0]])>

In [23]:
words_in_vocab = txt_vect.get_vocabulary() # all unique words
top_5 = words_in_vocab[:5]
top_5

['', '[UNK]', 'i', 'to', 'the']

In [24]:
bottom_5 = words_in_vocab[-5:]
bottom_5

['reschedule', 'rerun', 'reripped', 'rereading', 'reread']

## Embedding Layer

In [25]:
embedding = keras.layers.Embedding(input_dim=MAX_VOCAB_LEN,
                             output_dim=128,
                             input_length=MAX_LEN)

embedding

In [26]:
# Random sentences
random_sent = random.choice(train_sentences)
print(f"OG Sentence: {random_sent}")


OG Sentence: Screw the reviews, I thought Wolverine was awesome. But not enough Dominic Monaghan for my liking.


In [27]:
# Embed the random sentence (Dense Vector of fixed size)

sample_embed = embedding(txt_vect([random_sent]))
sample_embed


<tf.Tensor: shape=(1, 13, 128), dtype=float32, numpy=
array([[[ 0.01899043, -0.03920998, -0.03570122, ..., -0.04622573,
          0.04168886,  0.03316842],
        [-0.03810155, -0.00868433, -0.00590051, ..., -0.03730354,
          0.03825575, -0.04586705],
        [ 0.0344837 , -0.02609521,  0.04953663, ..., -0.04423419,
         -0.04848344, -0.0020285 ],
        ...,
        [-0.02090742, -0.03901745,  0.01449107, ..., -0.0127893 ,
         -0.03883722, -0.02554003],
        [ 0.03909126,  0.00890004,  0.03305476, ...,  0.04557728,
          0.03846404,  0.03461163],
        [ 0.03909126,  0.00890004,  0.03305476, ...,  0.04557728,
          0.03846404,  0.03461163]]], dtype=float32)>

## Baseline Model (SkLearn)

In [28]:

## Baseline Model (SkLearn)

model_1 = Pipeline([
  ("tfidf", TfidfVectorizer()),
  ("clf", MultinomialNB())
])

In [29]:
# Fitting model
model_1.fit(train_sentences,
            train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [38]:
# Evaluating the model
baseline_score = model_1.score(val_sentences, val_labels)
baseline_score
print(f"{baseline_score*100:.2f}%")

86.44%


In [31]:
# Making predictions
baseline_preds = model_1.predict(val_sentences)
baseline_preds

array([1, 1, 0, ..., 0, 0, 0])

# Simple Dense model

In [45]:
# Functional API (since we have custom layers already created)

inputs = layers.Input(shape=(1,), dtype="string") # 1D inputs
# Vectorization layer
x = txt_vect(inputs)
# Embedding layer
x = embedding(x)
# Average pooling
x= layers.GlobalAveragePooling1D()(x)
# Dense output
outputs = layers.Dense(1, activation="sigmoid")(x)
# Model creation
model_2 = tf.keras.Model(inputs, outputs, name="model_2_simple_dense")


In [46]:
model_2.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [47]:
model_2.summary()

Model: "model_2_simple_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 13)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 13, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,28

In [48]:
hist_2 = model_2.fit(train_sentences,
            train_labels,
            epochs=10,
            validation_data=(val_sentences, val_labels))

Epoch 1/10
461/461 [==============================] - 9s 12ms/step - loss: 0.5421 - accuracy: 0.7602 - val_loss: 0.3982 - val_accuracy: 0.8485
Epoch 2/10
461/461 [==============================] - 4s 9ms/step - loss: 0.3357 - accuracy: 0.8661 - val_loss: 0.3330 - val_accuracy: 0.8552
Epoch 3/10
461/461 [==============================] - 4s 9ms/step - loss: 0.2639 - accuracy: 0.9018 - val_loss: 0.3237 - val_accuracy: 0.8613
Epoch 4/10
461/461 [==============================] - 4s 9ms/step - loss: 0.2225 - accuracy: 0.9166 - val_loss: 0.3281 - val_accuracy: 0.8522
Epoch 5/10
461/461 [==============================] - 4s 9ms/step - loss: 0.1921 - accuracy: 0.9313 - val_loss: 0.3433 - val_accuracy: 0.8473
Epoch 6/10
461/461 [==============================] - 4s 9ms/step - loss: 0.1695 - accuracy: 0.9408 - val_loss: 0.3646 - val_accuracy: 0.8491
Epoch 7/10
461/461 [==============================] - 5s 10ms/step - loss: 0.1516 - accuracy: 0.9460 - val_loss: 0.3879 - val_accuracy: 0.8345
Epoc

In [50]:
loss, accuracy = model_2.evaluate(val_sentences, val_labels, verbose=1)
print(f"Loss: {loss*100:.2f}\nAccuracy: {accuracy*100:.2f}")

52/52 [==============================] - 0s 6ms/step - loss: 0.4825 - accuracy: 0.8351
Loss: 48.25
Accuracy: 83.51
